In [1]:
from util import *

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']
Add path ../../..
Connected DB - conn
Connected oraDW_ANALYTICS - conn_aly


In [2]:
import os
import numpy as np

In [3]:
import matplotlib.image as mpimg

In [4]:
from IPython.display import HTML

# NAMESPACE

In [5]:
class NAMESPACE:
    # Base values
    TABLE_NAME = 'DWA_STMT_EBANK'
    SEL_SAMPLE = 'SAMPLE1_SUB3'
    
    # Important to look
    # QUERY = f"""SELECT X.* FROM DW_ANALYTICS.{TABLE_NAME} X JOIN CINS_TMP_ALY_CUST_{SEL_SAMPLE} SAM ON X.CUSTOMER_ID = SAM.CUSTOMER_CDE"""
    QUERY = f"""with 
    t1 as (
        select * from DW_ANALYTICS.DWA_STMT_EBANK
    ),
    t2 as (
        select *
        from CINS_TMP_ALY_CUST_SAMPLE1_SUB3
        where 1=1
        and customer_cde not like '%#%'
        and customer_cde not like '%PAY%'
        and customer_cde not like '%INS%'
    ),
    t3 as (
        select t1.* from t1
        join t2 on t1.CUSTOMER_ID = t2.CUSTOMER_CDE
        JOIN DW_ANALYTICS.TRANSACTION_CODE TC ON t1.TRANSACTION_CODE = TC.TRANSACTION_CODE
        where t1.PRODUCT_CATEGORY LIKE '10__'
        AND TC.INITIATION = 'CUSTOMER'
    )
    select * from t3"""
    
    # Derived values
    data_fp = f'./data/{TABLE_NAME}'
    data_sample_nm = os.path.join(data_fp, SEL_SAMPLE)
    IMG_FP = f'./img/{TABLE_NAME}'
    img = f'./img/{TABLE_NAME}/{SEL_SAMPLE}'
    
    def show():
        atts = dir(NAMESPACE)
        for a in atts:
            if not a.startswith('__'):
                print(a)
    
    @classmethod
    def init(cls):
        if not os.path.exists(NAMESPACE.data_fp):
            os.makedirs(NAMESPACE.data_fp)
        if not os.path.exists(NAMESPACE.img):
            os.makedirs(NAMESPACE.img)

In [6]:
NAMESPACE.QUERY

"with \n    t1 as (\n        select * from DW_ANALYTICS.DWA_STMT_EBANK\n    ),\n    t2 as (\n        select *\n        from CINS_TMP_ALY_CUST_SAMPLE1_SUB3\n        where 1=1\n        and customer_cde not like '%#%'\n        and customer_cde not like '%PAY%'\n        and customer_cde not like '%INS%'\n    ),\n    t3 as (\n        select t1.* from t1\n        join t2 on t1.CUSTOMER_ID = t2.CUSTOMER_CDE\n        JOIN DW_ANALYTICS.TRANSACTION_CODE TC ON t1.TRANSACTION_CODE = TC.TRANSACTION_CODE\n        where t1.PRODUCT_CATEGORY LIKE '10__'\n        AND TC.INITIATION = 'CUSTOMER'\n    )\n    select * from t3"

In [7]:
NAMESPACE.init()

In [8]:
NAMESPACE.show()

IMG_FP
QUERY
SEL_SAMPLE
TABLE_NAME
data_fp
data_sample_nm
img
init
show


In [9]:
stat_opt = [
            'count',
            'mean',
            'median',
            'min',
            'max',
            'std',
]
ordered_stat_opt = [[i, i + '_pct_chg'] for i in stat_opt]
ordered_stat_opt = [j for i in ordered_stat_opt for j in i]

# Functions

In [10]:
def aly_datetime(df, date_col, cde_col, resample='1Y', outlier_date='1930-01-01', timedelta='365 days',lvl=3):
    display(HTML(f'<h{lvl}>{date_col}</h{lvl}>'))
    if cde_col is None:
        tmp_df = df[date_col].copy()
        tmp_df[date_col] = pd.to_datetime(tmp_df[date_col], errors='coerce')
        tmp_df[date_col] = df[date_col].astype('datetime64[D]')
        tmp = tmp_df[tmp_df[date_col] > outlier_date]
        plot_df = tmp.set_index(date_col).resample(resample).nunique()
    else:
        tmp_df = df[[date_col, cde_col]].copy()
        tmp_df[date_col] = pd.to_datetime(tmp_df[date_col], errors='coerce')
        tmp_df[date_col] = df[date_col].astype('datetime64[D]')
        tmp = tmp_df[tmp_df[date_col] > outlier_date]
        plot_df = tmp.set_index(date_col)[cde_col].resample(resample).nunique()
    
    fig, axs = plt.subplots(1,2, figsize=(15, 5))
    plot_df.plot(kind='bar', rot=70, ax=axs[0])
    up2now = ((datetime(2023,11,1) - tmp[date_col]) / pd.Timedelta(timedelta)).apply(float)
    sns.histplot(up2now, kde=True, ax=axs[1])
    plt.show()

In [11]:
def default_func(x):
    return x

In [12]:
def remove_mode_value(x):
    return x[~x.isin(x.mode().values)]

In [13]:
def logx(x):
    return np.log(x)

In [14]:
def plot_multi_hist_aly(df,cols=[],funcs=[default_func, remove_mode_value, logx], lvl=3):
    def df_stat_after_transformed(df_trans):
        def pct_change(series):
            first_e = series.iloc[0]
            pct_chg = (series - first_e) / first_e * 100
            pct_chg.iloc[0] = 100.
            return pct_chg
        stat_infos = []
        for data_trans_series in df_trans:
            # Get stats info
            stat_info = []
            for s in stat_opt:
                if s == 'count':
                    stat_info.append(data_trans_series.count())
                elif s == 'mean':
                    stat_info.append(data_trans_series.mean())
                elif s == 'median':
                    stat_info.append(data_trans_series.median())
                elif s == 'min':
                    stat_info.append(data_trans_series.min())
                elif s == 'max':
                    stat_info.append(data_trans_series.max())
                elif s == 'std':
                    stat_info.append(data_trans_series.std())
            stat_infos.append(stat_info)
        col_stat = stat_opt
        index_stat = [f.__name__ for f in funcs]
        stat_df = pd.DataFrame(data=stat_infos, columns=col_stat, index=index_stat)
        for c in stat_df.columns:
            stat_df[f'{c}_pct_chg'] = pct_change(stat_df[c])
        stat_df = stat_df[ordered_stat_opt]
        result_df = stat_df.T
        return result_df
    
    nrows, ncols = len(cols), len(funcs)
    for i, c in enumerate(cols):
        display(HTML(f'<h{lvl}>{c}</h{lvl}>'))
        if c in df.columns:
            data_trans = []
            for f in funcs:
                pre_data_tran = data_trans[-1] if len(data_trans) > 0 else df[c]
                trans_data = f(pre_data_tran)
                data_trans.append(trans_data)
            # plot section 
            fig, axs = plt.subplots(1,ncols, figsize=(4* ncols, 4 ))
            for j, d in enumerate(data_trans):
                sel_ax = axs[j]
                sel_ax.set_title(f'{cols[i]}\n{funcs[j].__name__}')
                d.replace([np.inf, -np.inf], np.nan).plot(kind='hist',bins=100,ax=sel_ax)
            pd.options.display.float_format = '{:.3f}'.format
            display(df_stat_after_transformed(data_trans))
            plt.tight_layout()
            plt.savefig(os.path.join(NAMESPACE.img, f'{c}.jpg'), dpi=500)
            plt.show()
            print('\n\n\n')

In [15]:
def aly_cat_col(df,cat_cols,k=10, lvl=3):
    for cat_col in cat_cols: 
        display(HTML(f'<h{lvl}>{cat_col}</h{lvl}>'))
        fig,axs=plt.subplots(1,2,figsize=(10,5))
        display(df[cat_col].value_counts())
        df[cat_col].value_counts()[:k].plot(kind='bar', rot=80,ax=axs[0])
        df[cat_col].value_counts()[:k].plot(kind='pie', autopct='%.2f%%',ax=axs[1])
        plt.savefig(os.path.join(NAMESPACE.img, f'{cat_col}.jpg'), dpi=500)
        plt.show()

In [16]:
def download_or_reload(saved_file):
    if not os.path.exists(saved_file):
        query = NAMESPACE.QUERY
        cur.execute(query)
        result = cur.fetchall()
        column_names = [c[0] for c in cur.description]
        df = pd.DataFrame(result, columns=column_names)
        df.to_pickle(saved_file)
    df = pd.read_pickle(saved_file)
    print(f'Len {len(df)}')
    return df

In [17]:
def gen_info_stats(df):
    def info_stats(c):
        dtype = df[c].dtype
        n = len(df)
        null_pct = (n - df[c].count()) / n
        nunique = df[c].nunique()
        nunique_pct = nunique / n
        unique_values = df[c].unique()[:3]
        return c, null_pct, dtype, nunique, nunique_pct, unique_values

    data_stats = []
    for c in df.columns:
        data_stats.append(info_stats(c))
    data_stats = pd.DataFrame(data=data_stats,columns=['Column', 'Null Pct', 'Dtype', 'Nunique', 'Nunique Pct', 'Sample Unique Value'])
    return data_stats

# Sample1-Sub*

In [18]:
NAMESPACE.QUERY

"with \n    t1 as (\n        select * from DW_ANALYTICS.DWA_STMT_EBANK\n    ),\n    t2 as (\n        select *\n        from CINS_TMP_ALY_CUST_SAMPLE1_SUB3\n        where 1=1\n        and customer_cde not like '%#%'\n        and customer_cde not like '%PAY%'\n        and customer_cde not like '%INS%'\n    ),\n    t3 as (\n        select t1.* from t1\n        join t2 on t1.CUSTOMER_ID = t2.CUSTOMER_CDE\n        JOIN DW_ANALYTICS.TRANSACTION_CODE TC ON t1.TRANSACTION_CODE = TC.TRANSACTION_CODE\n        where t1.PRODUCT_CATEGORY LIKE '10__'\n        AND TC.INITIATION = 'CUSTOMER'\n    )\n    select * from t3"

In [19]:
df = download_or_reload(NAMESPACE.data_sample_nm)

: 

: 

: 

In [20]:
df

,BANK_ID,STMT_ENTRY_ID,ACCOUNT_NUMBER,COMPANY_CODE,AMT_LCY,TRANSACTION_CODE,THEIR_REFERENCE,NARRATIVE,PL_CATEGORY,CUSTOMER_ID,...,CURR_NO,INPUTTER,DATE_TIME,AUTHORISER,CO_CODE,DEPT_CODE,AUDITOR_CODE,AUDIT_DATE_TIME,EOR,PROCESS_DATE
0,VNI,199685032962117.000001,070072495215,VN0012365,5500000.0,213,None,MBVCB.2403496542.316329.khanh chuyen tien.CT t...,None,10255531,...,5,10_DAVID.1,0901141547,10_DAVID.1,GB0010001,1,None,None,EOR,2022-12-04
1,VNI,200262548149037.020001,050130431582,VN0014473,-150000.0,213,None,MM2302535688 CASH IN-0913035998-30594165025 ...,None,16126482,...,5,10_DAVID.1,0901141547,10_DAVID.1,GB0010001,1,None,None,EOR,2022-12-04
2,VNI,199680915332010.000001,060248547191,VN0014113,1000000.0,213,None,PHAM THI NGOC ANH chuyen tien CKN 308479 - PHA...,None,14555128,...,5,10_DAVID.1,0901141547,10_DAVID.1,GB0010001,1,None,None,EOR,2022-12-04
3,VNI,199674648085012.000001,060248547191,VN0014113,-100000.0,213,None,MM2243366064 CASH IN-01634978882-28299149146 ...,None,14555128,...,5,10_DAVID.1,0901141547,10_DAVID.1,GB0010001,1,None,None,EOR,2022-12-04
4,VNI,200269079270937.020001,070112932656,VN0012392,9000000.0,213,None,MBVCB.2633328432.068667.DINH THI MINH THU chuy...,None,14252404,...,5,10_DAVID.1,0901141547,10_DAVID.1,GB0010001,1,None,None,EOR,2022-12-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1641310,VNI,197246844129466.000001,060213613769,VN0011169,-50000.0,213,None,MM1365788647 CASH IN-0931850606-19461413582 ...,None,12996591,...,5,10_DAVID.1,0901141547,10_DAVID.1,GB0010001,1,None,None,EOR,2022-12-04
1641311,VNI,197246495833097.030001,060213613769,VN0011169,-50000.0,213,None,MM1365794615 CASH IN-0931850606-19462800725 ...,None,12996591,...,5,10_DAVID.1,0901141547,10_DAVID.1,GB0010001,1,None,None,EOR,2022-12-04
1641312,VNI,197247335648111.000001,060213613769,VN0011169,-2000000.0,213,None,EWX151040284 IBFT Ngkhoa ck tien COD ...,None,12996591,...,5,10_DAVID.1,0901141547,10_DAVID.1,GB0010001,1,None,None,EOR,2022-12-04
1641313,VNI,197827596539369.000001,070099966988,VN0012452,3900000.0,213,None,NGUYEN MINH PHUC NGUYEN MINH PHUC chuyen tien ...,None,12840196,...,5,10_DAVID.1,0901141547,10_DAVID.1,GB0010001,1,None,None,EOR,2022-12-04


In [21]:
data_stats = gen_info_stats(df)
data_stats

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [ ]:
df.describe()

## EDA

Selected some col data to EDA, possible these col are good to go, (can be customized):
- Category data:
    - Unique Pct <= x (too many leading data can be identified easily)
    - Nunique > 1 (=1 doesn't mean anything)
- Numerical data: good to select all

In [ ]:
valid_info = None 
try: 
    valid_info = data_stats[((data_stats['Nunique Pct'] <= 0.3) & (data_stats['Nunique']>1) & (data_stats['Dtype']==object)) | ((data_stats['Dtype']!=object) & (data_stats['Nunique']>1))]
    valid_info = valid_info[~valid_info['Column'].isin([
        'CARD_KEY', 'CUSTOMER_CDE', 'CUSTOMER_KEY', 'SUB_BRANCH_KEY', 'SUB_BRANCH_CDE', 'COMPANY_KEY',
        'DAO_KEY', 'DAO_CDE', 'CREATE_DT', 'UPDATE_DT', 'NHAP_USERID', 'DUYET_USERID', 'NHANTHAMDINH_USERID',
        'RECOMMENDER_NO', 'RECOMMENDER_NAME', 'CARDHOLDER_NO', 'CRD_BASIC_CDE', 'NEW_CARDCODE', 'ID_NO', 'INDIVIDUAL_ACCT_ID',
        'TXN_ID', 'DAY_KEY', 'USER_KEY', 'USER_CDE', 'CORP_ID', 'ACCT_ID_FROM', 'SUB_BRANCH_KEY_FROM', 'SUB_BRANCH_CDE_FROM',
        'SUB_BRANCH_CDE_TO', 'SUB_BRANCH_KEY_TO', 'ACCT_ID_FROM', 'ACCT_NICKNAME_FROM', 'CONSUMER_CDE', 
        'PARENT_ID', 'USER_ID', 'USER_NAME', 'USER_DO', 'PRODUCT_ID', 'FROM_ACCT_ID', 'TO_ACCT_FULLNAME', 
        'PAYMENT_REF_NO', 'CARD_NO', 
    ])]
    
    display(valid_info.head(50))
except Exception as e:
    pass

In [ ]:
for i, r in valid_info.iterrows():
    if r['Dtype'] == object:
        aly_cat_col(df, [r['Column']])
    elif r['Dtype'] == 'datetime64[ns]':
        aly_datetime(df, r['Column'], 'CUSTOMER_CDE')
    else:
        plot_multi_hist_aly(df, [r['Column']])
    print('\n\n\n')

In [ ]:
df.columns, len(df.columns)

In [ ]:
valid_info.set_index('Column').index, len(valid_info['Column'].index)

In [ ]:
df.columns

## COMPARE SAMPLES - SUB1/2/3